# Exploring Toronoto Neighbourhoods

Install relevant packages

In [19]:
!pip3 install requests
!pip3 install bs4
!pip install requests
!pip3 install geocoder

import requests
from bs4 import BeautifulSoup

### Get Toronto Neighbourhoods from Wikipedia

In [20]:
# get html
page = requests.get('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&diff=945633050&oldid=942851379')
soup = BeautifulSoup(page.text, 'html.parser')

In [21]:
# import pandas
import pandas as pd

df = pd.read_html(str(soup.find_all('table')[1]))[0]
df.head()

### Remove unassigned postcodes

In [23]:
# Remove unassigned postcodes
df = df[df['Borough']!='Not assigned']
df.reset_index(inplace=True)
df.head()

,index,Postcode,Borough,Neighbourhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,Harbourfront
3,5,M6A,North York,Lawrence Heights
4,6,M6A,North York,Lawrence Manor


In [30]:
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(lambda x:', '.join(x)).to_frame().reset_index()
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Get latitude and longitudes for neighbourhoods

In [31]:
import geocoder

pc_list = []

for i in range(0,len(df)):
    address='{}, {}, {}, Toronto, Ontario'.format(df.at[i,'Postcode'],df.at[i,'Borough'],df.at[i,'Neighbourhood'])
    g = geocoder.arcgis(address)
    pc_list.append([df.at[i,'Postcode'],df.at[i,'Borough'],df.at[i,'Neighbourhood'], g.json['lat'], g.json['lng']])

In [32]:
# convert list to dataframe
pc_df = pd.DataFrame(data=pc_list,columns=['Postcode','Borough','Neighbourhood','Latitude','Longitude'])
pc_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785665,-79.158725
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765815,-79.175193
3,M1G,Scarborough,Woburn,43.768369,-79.217590
4,M1H,Scarborough,Cedarbrae,43.757320,-79.224550


In [33]:
df.shape

(103, 3)